**Các column của bảng quản lý nhà đất: address,price, area,category, extensions, project_legal(sổ hồng, sổ đỏ...), house_direction, posting_date, expire_date, average_price/1m2, population_density, AI_predicted**

Trang homedy.com

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
from concurrent.futures import ThreadPoolExecutor
import threading

lock = threading.Lock()

def append_to_json(data):
    filename = "/content/drive/MyDrive/Colab Notebooks/Data_homedy.json"
    with lock:  # Sử dụng lock để đảm bảo chỉ một luồng có thể ghi vào tệp JSON tại một thời điểm
        with open(filename, 'a', encoding='utf-8') as file:
            json.dump(data, file, indent=4, ensure_ascii=False)

def get_scrapeops_data(url):
    response = requests.get(f'https://homedy.com{url}')
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        target_div_root = soup.find('div', class_='content')
        address = target_div_root.find('div',class_ ='address')
        description = target_div_root.find('div',class_ = 'description readmore')
        price_area = target_div_root.find('div',class_='product-short-info')
        date_info = target_div_root.find('div',class_='product-info')
        posting_date = date_info.find_all('p', class_='code')[0].text.strip()
        expire_date = date_info.find_all('p', class_='code')[1].text.strip()
        status = date_info.find_all('p', class_='code')[2].text.strip()
        price_info = price_area.find('div', class_='short-item').find('strong').text.strip()
        area_info = price_area.find_all('div', class_='short-item')[1].find('strong').text.strip()
        address_parts = [span.get_text(strip=True) for span in address.find_all('span')]
        data = {
            "title": address.find('a').get_text(strip=True) if address else None,
            "address": ', '.join(address_parts),
            "description": description.text.strip() if description else None,
            "price": price_info,
            'area': area_info ,
            'posting_date': posting_date,
            'expire_date': expire_date,
            'status': status
        }
        return data

def process_page(page_number):
    url = f'https://homedy.com/ban-nha-dat-ha-noi/p{page_number}'
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        target_links = soup.find_all('a', class_='thumb-image', href=True)
        all_data = []
        for link in target_links:
           data = get_scrapeops_data(link['href'])
           all_data.append(data)
        append_to_json(all_data)
    else:
        print(f"Yêu cầu không thành công cho trang {url}: {response.status_code}")

def main():
    with ThreadPoolExecutor(max_workers=100) as executor:
        executor.map(process_page, range(1, 200))

if __name__ == "__main__":
    main()


Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p95: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p94: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p102: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p101: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p99: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p100: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p93: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p98: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p96: 524
Yêu cầu không thành công cho trang https://homedy.com/ban-nha-dat-ha-noi/p97: 524


Trang dothi.net

In [ ]:
from bs4 import BeautifulSoup
import requests
import json
from concurrent.futures import ThreadPoolExecutor
import threading

lock = threading.Lock()

def append_to_json(data):
    filename = "/content/drive/MyDrive/Colab Notebooks/Data_dothi.json"
    with lock:  # Sử dụng lock để đảm bảo chỉ một luồng có thể ghi vào tệp JSON tại một thời điểm
        with open(filename, 'a', encoding='utf-8') as file:
            json.dump(data, file, indent=4, ensure_ascii=False)

def get_scrapeops_data(url):
    response = requests.get(f'https://dothi.net{url}')
    if response.status_code == 200:
        try:
            soup = BeautifulSoup(response.content, 'html.parser')
            target_div_root = soup.find('div', class_='product-detail')
            address = target_div_root.find('div', class_='pd-location')
            description = target_div_root.find('div', class_='pd-desc')
            price_area = target_div_root.find('div', class_='pd-price')
            price_info = price_area.find('span', class_='spanprice').text.strip()

            # Trích xuất thông tin diện tích
            area_info = price_area.find_all('span')[1].text.strip()
            detail_div = target_div_root.find('div', class_='pd-dacdiem')
            detail_table = detail_div.find('table')
            rows = detail_table.find_all('tr')
            data = {}
            for row in rows:
                cells = row.find_all('td')
                if len(cells) == 2:
                    key = cells[0].text.strip()
                    value = cells[1].text.strip()
                    data[key] = value

            posting_date = data.get('Ngày đăng tin')
            expire_date = data.get('Ngày hết hạn')
            status = None  # You need to fill this according to your source
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

        data = {
            "title": address.find('a').get_text(strip=True) if address else None,
            "address": address.get_text().split('-', 1)[1].strip() if '-' in address.get_text() else '',
            "description": description.text.strip() if description else None,
            "price": price_info,
            'area': area_info,
            'posting_date': posting_date,
            'expire_date': expire_date,
            'status': status
        }
        return data

def process_page(page_number):
    url = f'https://dothi.net/nha-dat-ban-ha-noi/p{page_number}.htm'
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        target_links = soup.select('a.vip2, a.vip5')
        all_data = []
        for link in target_links:
            data = get_scrapeops_data(link['href'])
            all_data.append(data)
        append_to_json(all_data)
    else:
        print(f"Yêu cầu không thành công cho trang {url}: {response.status_code}")

def main():
    with ThreadPoolExecutor(max_workers=100) as executor:  # Số luồng tối đa là 100
        executor.map(process_page, range(1, 400))

if __name__ == "__main__":
    main()